# Tutorial

Let's take a look at how to use Trident.

## Setup

To use Trident, you need to install it yourself, because it's not yet registered with PyPI. It will also install the latest Triton. Trident can be installed with the following commands.


In [ ]:
!git clone https://github.com/kakaobrain/trident && pushd trident && bash install_package.sh

If you need to update Trident, you can do so with the following commands.

In [ ]:
!pushd trident && git pull && bash install_package.sh

## Usage

If you are familiar with Pytorch, using Trident will be a piece of cake, as Trident provides the same level of abstraction as PyTorch. The following code is taken from examples of PyTorch.

In [ ]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.LSTM(input_size=28, hidden_size=64, batch_first=True)
        self.norm = nn.InstanceNorm1d(64)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, input):
        # Shape of input is (batch_size,1, 28, 28)
        # converting shape of input to (batch_size, 28, 28)
        # as required by RNN when batch_first is set True
        input = input.reshape(-1, 28, 28)
        output, hidden = self.rnn(input)

        # RNN output shape is (seq_len, batch, input_size)
        # Get last output of RNN
        output = output[:, -1, :]
        output = self.norm(output)
        output = self.dropout1(output)
        output = self.fc1(output)
        output = F.relu(output)
        output = self.dropout2(output)
        output = self.fc2(output)
        output = F.softmax(output, dim=1)
        return output


model = Net().to("cuda")
input = torch.randn(2000, 1, 28, 28).to("cuda")

print(model(input))

The first step is importing Trident.

In [108]:
import trident

The second step is to make sure Trident provides operations you want to use. Also you need to check the performance of operations. You can check the performance on [Trident's Wiki](https://github.com/kakaobrain/trident/wiki/Performance).

The third step is changing code in a model. All you need to do is changing the module's path. Because Trident provides the same level of abstraction as PyTorch.

- Change from `torch.nn` to `trident`
- Change from `torch.nn.functional` to `trident.function`

For example `torch.nn.InstanceNorm1d` can be changed to `trident.InstanceNorm1d`. Let's change `InstanceNorm1d`, `Dropout`, `relu`, `softmax` from PyTorch to Trident in `Net`.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.LSTM(input_size=28, hidden_size=64, batch_first=True)
        self.norm = trident.InstanceNorm1d(64)
        self.dropout1 = trident.Dropout(0.25)
        self.dropout2 = trident.Dropout(0.5)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, input):
        # Shape of input is (batch_size,1, 28, 28)
        # converting shape of input to (batch_size, 28, 28)
        # as required by RNN when batch_first is set True
        input = input.reshape(-1, 28, 28)
        output, hidden = self.rnn(input)

        # RNN output shape is (seq_len, batch, input_size)
        # Get last output of RNN
        output = output[:, -1, :]
        output = self.norm(output)
        output = self.dropout1(output)
        output = self.fc1(output)
        output = trident.function.relu(output)
        output = self.dropout2(output)
        output = self.fc2(output)
        output = trident.function.softmax(output, dim=1)
        return output


model = Net().to("cuda")
input = torch.randn(2000, 1, 28, 28).to("cuda")

print(model(input))

See how easy it is? Enjoy Trident!